In [ ]:
import networkx as nx
import numpy as np

def _runMEC(original_graph, masked_edges):
    '''Subfunction of `all_eulerian_circuits_in_multigraph`. This function 
       creates nested for-loops to find all Eulerian cycles in a graph.''' 
    
    # OK, let's get rid of all the edges that we already used:
    G = nx.restricted_view(original_graph, [], masked_edges)
    
    # The end point of the edge that we used last determines the current vertex:
    current_vertex = masked_edges[-1][1]
    
    # If the degree of the current vertex is zero we can't walk any further and
    # one of two things has happened: (i) We made a complete Eulerian circuit
    # or (ii) we made a circuit but came back to source to early.
    # In the second case, there are still edges left that we haven't used.
    if G.degree(current_vertex) == 0:
        # Let's make sure that we used up all edges in the original graph:
        if len(masked_edges)==original_graph.number_of_edges():
            # Yay, `masked edges` is an Eulerian circuit!
            yield(masked_edges)
    else:
        # The current vertex does not have degree zero, so we can still go from
        # the current vertex to other vertices. Again, each (remaining) edge at
        # the current vertex is a possible choice for completing the circuit:
        for e in G.edges(current_vertex, keys=True):
            # continue iterative process ...
            yield from _runMEC(original_graph, masked_edges+[e])      


def all_eulerian_circuits_in_multigraph(G, source=None):
    '''For an undirected multigraph (with labeled edges!) find all Eulerian 
       circuits that start in source vertex. Returns a generator for
       Eulerian circuits. Raises exception if the multigraph is not Eulerian.
       Using this function for large graphs is probably a bad idea.'''
    
    # Let's first check if the graph is Eulerian:
    if not nx.is_eulerian(G):
        raise nx.NetworkXError("G is not Eulerian.")
    
    # We replace G with a shallow copy of G. I don't know why this is needed
    G = G.copy()
    
    # If user did not specify source, set first vertex as source vertex
    if source is None:
        source = list(G.nodes())[0]
        
    # All edges at source node are possible first edges for an Eulerian circuit:
    for e in G.edges(source, keys=True):
        # Now we start the iterative process!
        yield from _runMEC(G, # original graph
                           [e]) # list of edges that we have already used

In [ ]:
##### DEMO #####

# Let's create an undirected multigraph with two edges between every pair of 
# vertices:

# First, choose the number of vertices in the complete graph. 
# (Don't get too crazy! Try 2 or 3. Maybe 4. For 4, you are going to get 
# 130560 Eulerian cycles. Who could possibly want that many Eulerian cycles?)
num_vertices = 3

# Now, we construct the multigraph:
G0 = nx.DiGraph(nx.complete_graph(num_vertices))
G = nx.MultiGraph()
for e in list(G0.edges()):
    G.add_edge(*e)
    
# Here are the edges of the multigraph. They have labels.
print(list(G.edges(keys=True)))

In [ ]:
# Let's get the Eulerian cycles
listEC = sorted(list(all_eulerian_circuits_in_multigraph(G)))

# How many different Eulerian circles did we find?
print(len(listEC))

# Let's look at them!
for ec in listEC:
    print(ec)